Todos:
    1. To prepare csv files using the pandas, you can use ''addingHeader2theresult.ipynb''
    2. To run the following code in order to obtain visual attention map per frame.

In [ ]:
import csv
import sys, os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from inputParameter import *
from IPython.core.debugger import Tracer; 

import scipy.ndimage as ndi
from scipy.spatial import distance

from itertools import cycle

from mpl_toolkits.mplot3d import Axes3D
from __future__ import division

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib._png import read_png
from matplotlib.cbook import get_sample_data
from matplotlib.lines import Line2D
# from scipy.misc import imsave
import imageio
%matplotlib inline  

global Fixations_person

class Point:
    def __init__(self, posX, posY, registeredTimes):
        self.ClId = 0
        self.times = np.array(registeredTimes)
        self.pos = posX * posY
        self.posX= posX
        self.posY= posY
    
    def ChangeClId(self, ClId):
        self.ClId = ClId


%matplotlib inline

np.set_printoptions(threshold='nan')

#sns.set(color_codes=True)

plt.close('all')

width = 20
height = 20
time= '10'

if not os.path.exists(resFolder):
    os.makedirs(resFolder)

df = pd.read_csv( userStudyCSV + ".csv" )

persons_array = list(df['parName'].unique())
sphRad       = list(df['sphRad'].unique())
numSeq       = list(df['seqID'].unique())

src_file = [ sequence(x) for x in ['8K_MPEG', '4K_MPEG_v2'] ]
src_file = src_file[0] + src_file[1]

In [ ]:
def ExpandFixation(CenteredFixations, radio):
    
    expandedFixations =np.zeros_like(CenteredFixations) 
    
    for x in range(501):
        for y in range(1001):
            if CenteredFixations[x,y] == 1: # Means it's a cluster "center"
                
                # Expand the fixation radio pixels
                for i in range(radio):
                    for j in range(radio):
    
                        x1 = x + i
                        x2 = x - i
                        y1 = y + j
                        y2 = y - j
                    
                        try:
                            expandedFixations[x1, y] = 1
                            expandedFixations[x2, y] = 1
                            expandedFixations[x, y1] = 1
                            expandedFixations[x, y2] = 1
                            expandedFixations[x1, y1] = 1
                            expandedFixations[x1, y2] = 1
                            expandedFixations[x2, y1] = 1
                            expandedFixations[x2, y2] = 1
                            
                        except: # errors on the size of the matrix
                            pass
                        
    return expandedFixations

In [ ]:
def ExpandCluster(setOfPoints, point, eps, minPts, ClId): # DBSCAN method 
    seeds = RegionQuery(setOfPoints, point, eps) # returns the eps-neighborhood of point
    
    if len(seeds) < minPts :
        point.ChangeClId = 1 # Meaning Noise
        return False
    else:
        Fixations_person[point.posX, point.posY] = 1 # Getting only the center point of the fixation
        for i in range(len(seeds)):
            Fixations_person[seeds[i].posX, seeds[i].posY] = 1 # not only the center
            seeds[i].ChangeClId(ClId)
            
        ''' seeds.remove(point)
        while (len(seeds) != 0):
            currentPoint = seeds[0]
            if( currentPoint.posX > 9 and  currentPoint.posX < 491 and currentPoint.posY > 9 and currentPoint.posY < 991):
                result = RegionQuery(setOfPoints, currentPoint, eps)
                if len(result) >= minPts :
                    for extraPoint in result:
                        if (extraPoint.ClId == 0): # Unclassified 
                            extraPoint.ChangeClId(ClId)
                            seeds.append(extraPoint)
                        elif (extraPoint.ClId == 1): # Noise
                            extraPoint.ChangeClId(ClId) 
                seeds.remove(currentPoint) '''
        
        return True

In [ ]:
def RegionQuery(setOfPoints, point, eps):
    seeds = []
    
    # We append the point as many times as it appears, it should count for the fixations
    pointTimes = point.times # This is an np.array :)
    for k in range(1, len(pointTimes)):
        seeds.append(Point(point.posX, point.posY, pointTimes[k])) 
    
    
    for i in range(eps):
        for j in range(eps):
            
            # All possible combinations in an eps distance from the point
            x1=point.posX + i
            x2=point.posX - i
            y1=point.posY + j
            y2=point.posY - j
            
            # We append the point as many times as it appears, it should count for the fixations

            if i != 0 or j != 0 :
                times_x1_y1 = np.array(setOfPoints[x1, y1])
                if times_x1_y1.size > 1:
                    for k in range( 1, len(times_x1_y1)):
                        seeds.append(Point(x1, y1, times_x1_y1[k]))

                times_x1_y2 = np.array(setOfPoints[x1, y2])
                if times_x1_y2.size > 1:
                    for k in range( 1, len(times_x1_y2)):
                        seeds.append(Point(x1, y2, times_x1_y2[k]))

                times_x2_y1 = np.array(setOfPoints[x2, y1])
                if times_x2_y1.size > 1:
                    for k in range( 1, len(times_x2_y1)):
                        seeds.append(Point(x2, y1, times_x2_y1[k]))

                times_x2_y2 = np.array(setOfPoints[x2, y2])
                if times_x2_y2.size > 1:
                    for k in range( 1, len(times_x2_y2)):
                        seeds.append(Point(x2, y2, times_x2_y2[k])) 
                    
    
    return seeds

In [ ]:
array_x_extended = np.array([])
array_y_extended = np.array([])
distVisited =[]

h    = np.array(src_file).shape
w    = len(persons_array)

imageName_array      = [ src_file[x]['latex'] for x in range(0,h[0]) ]
distVisited_matrix   = [ [0 for x in range(w)] for y in range(0,h[0]) ]
imageTime            = []
cumDistVisited_array = []

numFixPointsTotal    = {}
xxx                  = {}

for image_index, image in enumerate(src_file):
    savedImageName = imageName_array[image_index]
    imageName      = imageName_array[image_index]
    
    cumDistVisited = 0
    cumDistVisited_index = 0
    
    GroupedFixations = np.zeros((501, 1001))

    condition = (df['seqID']==image['sal_playID'])
    dataPoints = df[condition]
    numFixPointsTotal[imageName] = {}
    #         Tracer()()
    #     for fN in range(60,80):  
    #     for fN in range(20,30):  
    #     for fN in range(40,50): 
    
    for fN in range(1, int(dataPoints['seqDur'].value_counts()[dataPoints['seqDur'].value_counts() == max( dataPoints['seqDur'].value_counts() )].keys()[0])):
        numFixPointsTotal[imageName][fN] = {}
#       consider the time period  
        condition = (dataPoints['seqTime']<fN) & (dataPoints['seqTime']>fN-1)
        dataPoint = dataPoints[condition]
        numFixPoints = np.zeros((501, 1001))
        
        print( 'Frame # ' + str(fN) )
#         Tracer()()
    
        for person_idx, person in enumerate(persons_array) :
            Fixpoints = []

            print(person,image['sal_playID'], image['name'])
            condition = (dataPoint['seqID']==image['sal_playID']) & (dataPoint['parName']==person)
            dataP = dataPoint[condition]
            
#             if image['sal_playID'] == 14:
#                 condition = (dataP['seqTime']<=10) & (dataP['seqTime']>=0)
#                 dataP1 = dataP[ condition ]
#                 condition = (dataP['seqTime']<=30) & (dataP['seqTime']>=20)
#                 dataP2 = dataP[ condition ]
#                 condition = (dataP['seqTime']<=50) & (dataP['seqTime']>=40)
#                 dataP3 = dataP[ condition ]
#                 condition = (dataP['seqTime']<=80) & (dataP['seqTime']>=60)
#                 dataP4 = dataP[ condition ]
#                 frames = [dataP1, dataP2, dataP3, dataP4]
#                 dataP = pd.concat(frames)
#                 or ((dataP['seqTime']<=30) and (dataP['seqTime']>=20))
#                 || ((dataP['seqTime']<=50) & (dataP['seqTime']>=40)) \
#                 || ((dataP['seqTime']<=80) & (dataP['seqTime']>=60))
                
#                 Tracer()()
                      
#             dataP = dataP[dataP['seqTime']<=40]
                                    
            RegisteredPoints_person= np.zeros((501, 1001), dtype=object) # Created to have a heat map per person
            ProbMatrix_person= np.zeros((501, 1001), dtype=np.int) # Created to have a heat map per person
        
            # don't consider the first fixation (1s)
            # dataPoints = dataPoints[40:] #ana=40
            
            array_x    = round(dataP.us2DVPMu*1000).astype(int)
            array_y    = round(dataP.us2DVPMv*500).astype(int)

#             array_x    = round(dataP.us2DVPMu*100 + 500).astype(int)
#             array_y    = round(dataP.us2DVPMv*100 + 250).astype(int)

#             array_x.append(int((np.around(float(row[3]), decimals=2)*100) + 500)) # Depends on the size of the equirectangular image
#             array_y.append(int((np.around(float(row[4]), decimals=2)*100) + 250))
            
            
            array_time = dataP['seqTime']
        
            max_x =  array_x.max()
            min_x =  array_x.min()
            
            
            #*********************************************************************************
            # Calculate distance visited by the users
            #*********************************************************************************
                
            array_xSorted = np.sort(array_x)
            maxDiff = (999 - array_xSorted[-1]) + (array_xSorted[0] - 0)
            pointA = array_xSorted[0] 
            pointB = array_xSorted[-1]
            entered = 0
            

            for i in range(len(array_xSorted) - 1):
                diff= array_xSorted[i+1] - array_xSorted[i]
                if maxDiff < diff:
                    maxDiff = diff
                    pointA = array_xSorted[i]
                    pointB = array_xSorted[i+1]
                    entered = 1;
        
            if len(persons_array) > 1:
                #print(image_index,person_idx, len(distVisited_matrix), len(distVisited_matrix[0]) )
            
                if cumDistVisited_index == len(persons_array) - 1: # means next will be a new image :)
                    if (entered == 1) :
                        cumDistVisited = cumDistVisited + (pointA + (1000 - pointB))
                        distVisited_matrix[image_index][person_idx] = pointA + (1000 - pointB)
                    else:
                        cumDistVisited = cumDistVisited + (pointB - pointA)
                        distVisited_matrix[image_index][person_idx] = (pointB - pointA)
                    cumDistVisited_array = np.append(cumDistVisited_array, cumDistVisited)

            
                else:
                    cumDistVisited_index += 1
                
                    if (entered == 1) :
                        cumDistVisited = cumDistVisited + (pointA + (1000 - pointB))
                        distVisited_matrix[image_index][person_idx] = pointA + (1000 - pointB)
                    else:
                        cumDistVisited = cumDistVisited + (pointB - pointA)
                        distVisited_matrix[image_index][person_idx] = (pointB - pointA)

            if len(persons_array) == 1:
                imageTime = np.append(imageTime, firstTime ) # In order to have an ordered (regarding presentation time) plot 

                if (entered == 1) :
                    distVisited = np.append(distVisited, (pointA + (1000 - pointB)))
                        #print (savedImageName, 0, pointA, pointB, 1000, distVisited)
                else:
                    distVisited = np.append(distVisited,(pointB - pointA)) 
                    #print (savedImageName, pointA, pointB, distVisited)

            #*********************************************************************************
            # Assigns +1 in a position (x,y) detected
            #*********************************************************************************

            for i in range(len(array_x)):
#             print(array_x)
                x = array_x.iloc[i]
                y = array_y.iloc[i]
                ProbMatrix_person[y, x] += 1

                RegisteredPoints_person[y, x] = np.append(RegisteredPoints_person[y, x], array_time.iloc[i])
        

            #************************************************************************************************  
            #******************************************** DBSCAN ********************************************
            #                                       Looking for clusters
            #************************************************************************************************
        
            ClusterId = 2 # First ClusterId
        
            Fixations_person = np.zeros((501, 1001)) # Created to have a heat map per person
            for i in range(501):
                for j in range(1001):
                    point = Point(i, j, RegisteredPoints_person[i,j])
                
                    if point.times.size > 1: # Meaning, it was a visited point
                    
                        # point.ClId == 0 Means unclassified
                        xnew = i - 250 # I need the point centered in 0
                        minPts = 12
                        eps = np.rint(6 * (1 / (np.cos(xnew/250 * np.pi/2)))) # Distance - rounded to the closest integer

                        if (point.posX > eps and  point.posX < (501 - eps) and point.posY > eps and point.posY < (1001 - eps)):
                     
                            #print (xnew, (137/250 * np.pi/2), np.cos(0.86079638708), eps)
                       
                            if ExpandCluster(RegisteredPoints_person, point, int(eps), minPts, ClusterId):
                                ClusterId += 1

            #************************************************************************************************  
            #********************************* Post-processing per person ***********************************

            # Expand each fixation
            radio= 15
            Fixations_person = ExpandFixation(Fixations_person, radio)
            
#             PlotFixations_SaliencyMaps(Fixations_person, savedImageName, person, time)
#             PlotTemporalPath(array_x, array_y, array_time, savedImageName, person, time)
        
            # Add it to the grouped fixations
            GroupedFixations += Fixations_person
            numFixPoints += Fixations_person
            
        GroupedFixations = GroupedFixations/len(persons_array)

#     xxx[imageName] = np.nonzero(GroupedFixations)

#             %debug

#             numFixPointsTotal[imageName][fN][person] = np.count_nonzero(Fixations_person)
#             

#         %debug
    #     # Average group fixations        
        
    #     # Average group fixations
#         numFixPoints = numFixPoints/len(persons_array)
#     Tracer()()
#         numFixPoints = numFixPoints/len(persons_array)
        
#         Fixpoints.append(np.count_nonzero(numFixPoints))
# print(numFixPointsTotal)        
# %debug
#     Tracer()() #this one triggers the debugger

#         %debug 
        PlotFixations_SaliencyMaps(GroupedFixations, savedImageName, '0', fN)

#************************************************************************************************  
# Only in the case of one participant
#************************************************************************************************  
#if len(persons_array) == 1:

#     new_imageName_array =[]
#     new_distVisited = []

#     imageTime = np.array(imageTime)
#     sortedImageTime = np.sort(imageTime)

#     st = set(sortedImageTime)

#     for x in sortedImageTime: 
#         if x in imageTime:
#             index, = np.where( imageTime==x )
#             new_imageName_array.append(imageName_array[index])
#             new_distVisited.append(distVisited[index])

#     new_distVisited = np.array(new_distVisited)

#     # print (imageTime)
#     # print (sortedImageTime)

#     # print (imageName_array)
#     # print (new_imageName_array)
#     # print (distVisited)
#     # print (new_distVisited)

#     y = distVisited
#     x = range(len(imageName_array))
#     labels = imageName_array

#     y_new = new_distVisited
#     x_new = range(len(new_imageName_array))
#     labels_new = new_imageName_array


#     # Bar plot for individual Navigation Time

#     b1= plt.bar(x, y)
#     # You can specify a rotation for the tick labels in degrees or with keywords.
#     plt.ylim(0, 1000)
#     plt.xticks(x, labels, rotation='vertical')
#     # Pad margins so that markers don't get clipped by the axes
#     plt.margins(0.2)
#     # Tweak spacing to prevent clipping of tick-labels
#     plt.subplots_adjust(bottom=0.15)
#     plt.show()
#     plt.tight_layout()
#     # plt.savefig('C:/Users/Ana/Code/webvr-samples-master/panorama-viewer-master/Saliency/10s/NavigationTime_' + person +'.png')


#     plt.close('all')
#     plt.clf()

    # Bar plot for Ordered(presentation) individual Navigation Time

    # b2 = plt.bar(x_new, y_new)
    # #b2.[0].set_color('r')
    # plt.ylim(0, 1000)
    # plt.xticks(x_new, labels_new, rotation='vertical')
    # plt.margins(0.2)
    # plt.subplots_adjust(bottom=0.15)
    # #plt.show()
    # plt.tight_layout()
    # plt.savefig('C:/Users/Ana/Code/webvr-samples-master/panorama-viewer-master/Saliency/10s/OrderedNavigationTime_' + person +'.png')
#************************************************************************************************  
if len(persons_array) > 1:
    cumDistVisited_array = np.array(cumDistVisited_array)
    
    #y = np.array(cumDistVisited_array/len(persons_array)) # For mean value
    y = np.median(distVisited_matrix, 1) # For median value
    x = range(len(imageName_array))
    labels = imageName_array
    
    
        
    # calculate std 
    # For each image there are X participants
    std=[]
    for i in range(len(imageName_array)):
        indDist = np.array(distVisited_matrix[i])
        diff_std= np.power((y[i] - indDist), 2) 
        std = np.append(std, np.sqrt(np.sum(diff_std)/len(persons_array)) )
        
    y_mad = mad(distVisited_matrix, 1) # Median absolute deviation
    
#     print('std', std)
#     print('dist', y)
#     print ('mad', y_mad)

    plt.bar(x, y)
    plt.ylim(0, 1000)    
    plt.xticks(x, labels, rotation=30)
    plt.margins(0.2)
    plt.subplots_adjust(bottom=0.15)
    plt.tight_layout()
    
#     plt.savefig('CumulatedNavigationTime.png')

In [ ]:
from scipy.stats import norm                                                  
colors1 = [ [230/255,159/255,0], [204/255,121/255,167/255] , [0,158/255,115/255], [0,114/255,178/255], 
[213/255,94/255,0],[86/255,180/255,233/255], [204/255,121/255,167/255], [0,0,0]]

sizes = 1000000;

xax1 = [xxx['LRRH'][0][it] for it in range(len(xxx['LRRH'][0]))]
xax1 = xax1/max(xax1)*180
stdx1 = np.random.normal(np.mean(xax1), np.std(xax1), size=sizes)
stdx1_plot = np.linspace(min(stdx1), max(stdx1), 1000)    

yax1 = [xxx['LRRH'][1][it] for it in range(len(xxx['LRRH'][1]))]
yax1 = yax1/max(yax1)*360
stdy1 = np.random.normal(np.mean(yax1), np.std(yax1), size=sizes)
stdy1_plot = np.linspace(min(stdy1), max(stdy1), 1000)    

xax2 = [xxx['Gaslamp360'][0][it] for it in range(len(xxx['Gaslamp360'][0]))]
xax2 = xax2/max(xax2)*180
stdx2 = np.random.normal(np.mean(xax2), np.std(xax2), size=sizes)
stdx2_plot = np.linspace(min(stdx2), max(stdx2), 1000)    

yax2 = [xxx['Gaslamp360'][1][it] for it in range(len(xxx['Gaslamp360'][1]))]
yax2 = yax2/max(yax2)*360
stdy2 = np.random.normal(np.mean(yax2), np.std(yax2), size=sizes)
stdy2_plot = np.linspace(min(stdy2), max(stdy2), 1000)   

xax3 = [xxx['left_Driving360'][0][it] for it in range(len(xxx['left_Driving360'][0]))]
xax3 = xax3/max(xax3)*180
stdx3 = np.random.normal(np.mean(xax3), np.std(xax3), size=sizes)
stdx3_plot = np.linspace(min(stdx3), max(stdx3), 1000)    

yax3 = [xxx['left_Driving360'][1][it] for it in range(len(xxx['left_Driving360'][1]))]
yax3 = yax3/max(yax3)*360
stdy3 = np.random.normal(np.mean(yax3), np.std(yax3), size=sizes)
stdy3_plot = np.linspace(min(stdy3), max(stdy3), 1000)   

xax4 = [xxx['train_le'][0][it] for it in range(len(xxx['train_le'][0]))]
xax4 = xax4/max(xax4)*180
stdx4 = np.random.normal(np.mean(xax4), np.std(xax4), size=sizes)
stdx4_plot = np.linspace(min(stdx4), max(stdx4), 1000)    

yax4 = [xxx['train_le'][1][it] for it in range(len(xxx['train_le'][1]))]
yax4 = yax4/max(yax4)*360
stdy4 = np.random.normal(np.mean(yax4), np.std(yax4), size=sizes)
stdy4_plot = np.linspace(min(stdy4), max(stdy4), 1000)  

xax5 = [xxx['basketball'][0][it] for it in range(len(xxx['basketball'][0]))]
xax5 = xax5/max(xax5)*180
stdx5 = np.random.normal(np.mean(xax5), np.std(xax5), size=sizes)
stdx5_plot = np.linspace(min(stdx5), max(stdx5), 1000)    

yax5 = [xxx['basketball'][1][it] for it in range(len(xxx['basketball'][1]))]
yax5 = yax5/max(yax5)*360
stdy5 = np.random.normal(np.mean(yax5), np.std(yax5), size=sizes)
stdy5_plot = np.linspace(min(stdy5), max(stdy5), 1000)  

xax6 = [xxx['left_Dancing360'][0][it] for it in range(len(xxx['left_Dancing360'][0]))]
xax6 = xax6/max(xax6)*180
stdx6 = np.random.normal(np.mean(xax6), np.std(xax6), size=sizes)
stdx6_plot = np.linspace(min(stdx6), max(stdx6), 1000)    

yax6 = [xxx['left_Dancing360'][1][it] for it in range(len(xxx['left_Dancing360'][1]))]
yax6 = yax6/max(yax6)*360
stdy6 = np.random.normal(np.mean(yax6), np.std(yax6), size=sizes)
stdy6_plot = np.linspace(min(stdy6), max(stdy6), 1000)  

In [ ]:
bin = 500
import seaborn as sns
sns.reset_orig()
colors1 = [ [230/255,159/255,0], [204/255,121/255,167/255] , [0,158/255,115/255], [0,114/255,178/255], 
[213/255,94/255,0],[86/255,180/255,233/255], [204/255,121/255,167/255], [0,0,0]]

# sns.set()
# sns.reset_orig
sns.set_style('whitegrid')

leg = ['LRRH', 'Gaslamp360', 'leftDriving360', 'trainle', 'basketball', 'leftDancing360']
plt.figure(0)
# _ = plt.hist(xax1,bins=bin, color=colors1[0], normed=True, histtype='step', label= 'LRRH')       
# plt.figure(2)
# _ = plt.hist(xax2,bins=bin, color=colors1[1], normed=True, histtype='step', label= 'Gaslamp360')     
# plt.figure(3)
# _ = plt.hist(xax3,bins=bin, color=colors1[2], normed=True, histtype='step',label= 'leftDriving360')  
# plt.figure(4)
# _ = plt.hist(xax4,bins=bin, color=colors1[3], normed=True, histtype='step',label= 'trainle')         
# plt.figure(5)
# _ = plt.hist(xax5,bins=bin, color=colors1[4], normed=True, histtype='step', label='basketball')    
# plt.figure(6)
_ = plt.hist(xax6,bins=bin, color=colors1[5], normed=True, histtype='step', label='leftDancing360')         



# _ = plt.hist(stdx1,bins=bin, color=colors1[5], histtype='step')         
# _ = plt.hist(stdx2,bins=bin, color=colors1[4], histtype='step')         
# _ = plt.hist(stdx3,bins=bin, color=colors1[3], histtype='step')         
# _ = plt.hist(stdx4,bins=bin, color=colors1[2], histtype='step')         
# _ = plt.hist(stdx5,bins=bin, color=colors1[1], histtype='step')    
# _ = plt.hist(stdx6,bins=bin, color=colors1[0], histtype='step')         
plt.xlim(75, 180)

plt.legend( fontsize=24, markerscale=4, loc='upper left', fancybox=True, framealpha=0.5)

# plt.margins(0.02)

plt.tick_params(axis='x', which='major', labelsize=24)
plt.tick_params(axis='y', which='major', labelsize=20)
# plt.savefig('polar90_LRRH' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar90_Gaslamp360' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar90_leftDriving360' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar90_trainle' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar90_basketball' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
plt.savefig('polar90_leftDancing360' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')

# ax.set_thetamin(0)
# ax.set_thetamax(90)
# plt.tight_layout()

# Make 2% margin
# plt.margins(0.02)


# _ = plt.hist(yax1,bins=bin, color=colors1[5], histtype='step')         
# _ = plt.hist(yax2,bins=bin, color=colors1[4], histtype='step')         
# _ = plt.hist(yax3,bins=bin, color=colors1[3], histtype='step')         
# _ = plt.hist(yax4,bins=bin, color=colors1[2], histtype='step')         
# _ = plt.hist(yax5,bins=bin, color=colors1[1], histtype='step')    
# _ = plt.hist(yax6,bins=bin, color=colors1[0], histtype='step')         

# _ = plt.plot(stdy1_plot, norm.pdf(stdy1_plot, np.mean(stdy1) , np.std(stdy1)), color=colors1[5], label="pdf")            
# _ = plt.plot(stdy2_plot, norm.pdf(stdy2_plot, np.mean(stdy2) , np.std(stdy2)), color=colors1[4], label="pdf")            
# _ = plt.plot(stdy3_plot, norm.pdf(stdy3_plot, np.mean(stdy3) , np.std(stdy3)), color=colors1[3], label="pdf")            
# _ = plt.plot(stdy4_plot, norm.pdf(stdy4_plot, np.mean(stdy4) , np.std(stdy4)), color=colors1[2], label="pdf")            
# _ = plt.plot(stdy5_plot, norm.pdf(stdy5_plot, np.mean(stdy5) , np.std(stdy5)), color=colors1[1], label="pdf")            
# _ = plt.plot(stdy6_plot, norm.pdf(stdy6_plot, np.mean(stdy6) , np.std(stdy6)), color=colors1[0], label="pdf")  
# plt.margins(0.02)

In [ ]:
import numpy as np
import matplotlib.cm as cm
from matplotlib.pyplot import figure, show, rc
N = 250
sns.reset_orig()

sns.set()
# sns.reset_orig
# sns.set_style('whitegrid')

# force square figure and square axes looks better for polar, IMO
colors1 = [ [230/255,159/255,0], [204/255,121/255,167/255] , [0,158/255,115/255], [0,114/255,178/255], 
[213/255,94/255,0],[86/255,180/255,233/255], [204/255,121/255,167/255], [0,0,0]]

# data = yax1
# data = yax2
# data = yax3
data = yax4
# data = yax5
# data = yax6

# fig = figure(1)
# ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=False)

# _ = ax.hist( data, N, normed=True)

# ax.yaxis.grid(False)


# theta = np.arange(0.0, 2*np.pi, 2*np.pi/N)

counts, bins = np.histogram(data,N,normed=True)
countT = [counts[x] for x in range(len(counts))]
binT = [bins[x] for x in range(1,len(bins))]


theta = np.deg2rad(binT)
# radii = np.ones(len(binT))
fig = figure(2)

ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)

# width = np.pi/4*np.random.rand(len(theta))
# bars = ax.bar(theta, countT, 0, bottom=0.0 )

# ax.plot(theta, countT, "o")
width = 0.1
# ax.bar(theta, countT, width,color=colors1[0])
# ax.bar(theta, countT, width,color=colors1[1])
# ax.bar(theta, countT, width,color=colors1[2])
ax.bar(theta, countT, width,color=colors1[3])
# ax.bar(theta, countT, width,color=colors1[4])
# ax.bar(theta, countT, width,color=colors1[5])

ax.set_rticks([min(countT),max(countT)/2,max(countT)])

# plt.ylabel('PDF', fontsize=14)
# plt.xlabel('Number of fixations', fontsize=14)

ax.tick_params(axis='x', which='major', labelsize=24)
ax.tick_params(axis='y', which='major', labelsize=20)

# ax.set_thetamin(0)
# ax.set_thetamax(90)

# Make 2% margin
plt.margins(0.02)

# plt.tight_layout()

# plt.savefig('polar_LRRH' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar_Gaslamp360' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
plt.savefig('polar_trainle' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar_basketball' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')
# plt.savefig('polar_leftDancing360' + '.pdf', bbox_inches='tight', dpi=100, format='pdf')



# _ = ax.hist( np.deg2rad(yax5), N, normed=True)

# for r,bar in zip(radii, bars):
#     bar.set_facecolor( cm.jet(r/10.))
#     bar.set_alpha(0.5)

show()

In [ ]:
def ExpandCluster(setOfPoints, point, eps, minPts, ClId): # DBSCAN method 
    seeds = RegionQuery(setOfPoints, point, eps) # returns the eps-neighborhood of point
    
    if len(seeds) < minPts :
        point.ChangeClId = 1 # Meaning Noise
        return False
    else:
        Fixations_person[point.posX, point.posY] = 1 # Getting only the center point of the fixation
        for i in range(len(seeds)):
            Fixations_person[seeds[i].posX, seeds[i].posY] = 1 # not only the center
            seeds[i].ChangeClId(ClId)
            
        
        ''' seeds.remove(point)
        while (len(seeds) != 0):
            currentPoint = seeds[0]
            if( currentPoint.posX > 9 and  currentPoint.posX < 491 and currentPoint.posY > 9 and currentPoint.posY < 991):
                result = RegionQuery(setOfPoints, currentPoint, eps)
                if len(result) >= minPts :
                    for extraPoint in result:
                        if (extraPoint.ClId == 0): # Unclassified 
                            extraPoint.ChangeClId(ClId)
                            seeds.append(extraPoint)
                        elif (extraPoint.ClId == 1): # Noise
                            extraPoint.ChangeClId(ClId) 
                seeds.remove(currentPoint) '''
        
        return True
            


In [ ]:
# Plotting entropy
imageLabels = ['left_Dancing360', 'basketball', 'train_le', 'left_Driving360', 'LRRH', 'Gaslamp360']

N = len(imageLabels)
labels = ['$L_{2}$', '$L_{3}$', '$L_{4}$']
ind = np.arange(N)
T=2

colors = np.array(['steelblue', 'indianred', 'olivedrab','darkorange',  'magenta', 'cornflowerblue', 'magenta', 'salmon'])  
markers = np.array([">", "o", "s", "D", "d", "p"])

#eps= 3
# E = "[ 3.0488    2.6877    2.2423    2.1776    2.5205    2.3108    3.3392    2.4982    2.3369    2.4238    2.9503    3.3312    1.9963    3.8147    2.2981    2.2083    2.5597    2.3493    3.5168    2.6718    2.7052; 3.9530    3.7066    3.2083    2.4961    3.1128    2.7992    3.4991    3.4344    3.6343    2.9246    3.0463    3.2697    2.4265    3.5073    2.5982    2.9828    3.1914    3.2512    3.8075    3.3776    3.5976]"
# entropies = mag_func(E);

# # Plot of 10s
# fig=plt.figure()
# #plt.plot(ind, mean_fixations10[0], 'k--', label = 'mean')
# for i, color in zip(range(T), colors):
#     plt.plot(ind, entropies[i], marker = markers[i], markersize=7, label='%s' % labels[i])
# plt.xticks(ind, imageLabels, rotation=90)
# plt.ylabel('Entropy',fontsize=12)
# plt.legend()
import seaborn as sns

#eps= 6
#E = "[   3.5673    3.2574    2.6460    2.5520    2.7761    2.4864    3.6334    2.9789    2.9659    2.6756    3.1930    3.9104    2.6645    4.3792    2.9425    2.8185    3.3172    2.7304    3.8530    3.1441    3.2760; 4.4861    3.8957    3.6512    2.8788    3.4510    3.0521    3.6141    4.1068    3.9350    3.4237    3.2912    3.6864    2.8570    3.9464    2.9723    3.4069    3.7549    3.8661    4.4013    3.7184    4.0978]"

# E= "[2.5918    2.9617    3.7360    2.6941    2.8600    3.2613; 2.8788    2.9723    4.4861    3.6512    3.4510    3.2912]"

# entropies = mag_func(E);

# Plot of 10s
fig=plt.figure()
# plt.style.use('seaborn-whitegrid') 


l4 = [1.72763426285365,0.684606134535048,1.08042811807046,1.06375561779932,0.968161179462151,1.31244296509803]
le4 = [0.167728385461105,0.101025242243376,0.159406971737738,0.141765254248680,0.120520007087917,0.215329227431686]

l3 = [1.50764771001151,0.579946314804001,0.889012443756739,0.894691266829623,0.881159603660860,1.13440941040820]
le3 = [0.201033190083158,0.0846018352768310,0.143287191865047,0.148603626405014,0.132128898354163,0.195933885169943]

l2 = [1.20997886594809,0.425704011523369,0.700614939924144,0.648866010042241,0.710717659664374,0.917569772405377]
le2 = [0.226305102679320,0.0811932259550128,0.126189611850242,0.114715332581565,0.108624592082888,0.174632330214117]

#plt.plot(ind, mean_fixations10[0], 'k--', label = 'mean')
# for i in range(T):


plt.bar(ind-0.2, l2, width=0.2, color= colors[0], label='%s' % labels[0])
plt.bar(ind, l3, width=0.2, color= colors[1],  label='%s' % labels[1])
plt.bar(ind+0.2, l4, width=0.2, color= colors[2],  label='%s' % labels[2])

#     plt.scatter(  ind, entropies[i], marker = markers[i], color= colors[i], markersize=8, label='%s' % labels[i])

#     plt.scatter(  ind, entropies[i], marker = markers[i], color= colors[i], markersize=8, label='%s' % labels[i])

plt.xticks(ind, imageLabels, rotation=30,fontsize=16)
# plt.ylim(2,4.6)
plt.ylabel('Entropy',fontsize=14)
plt.legend(loc="best", fontsize=14, ncol=1)
#plt.gca().set_aspect('equal', adjustable='box')

plt.tick_params(axis='x', which='major', labelsize=14)
plt.tick_params(axis='y', which='major', labelsize=14)

plt.margins(0.01)
plt.tight_layout()
#plt.gca().axes.set_yticklabels([])


# fig.savefig('entropy' + '.pgf', bbox_inches='tight', dpi=300, format='pgf')

